In [79]:
import EegFunctions
import importlib
importlib.reload(EegFunctions)
from EegFunctions import *
%matplotlib qt

In [57]:
class Config:
    DATA_FILE =  r'C:\Users\Keerti\Downloads\MySolution\PlayBooks\data\15_N2pc_ICA_preprocessed_epoched.set'
    DEVICE = 'unicorn'  # 'muse' or 'unicorn'
    # SAMPLING_RATE = 250
    CHANNELS = ['TP9', 'AF7', 'AF8', 'TP10'] if DEVICE == 'muse' else ['Fz', 'C3', 'Cz', 'C4', 'Pz', 'PO7', 'Oz', 'PO8']
    ANALYSIS_CHANNELS = ['TP9', 'TP10'] if DEVICE == 'muse' else [ 'Pz','Cz']
    EPOCH_TMIN = -0.2
    EPOCH_TMAX = 0.8
    BASELINE = (-0.2, 0)
    HIGH_PASS = None
    LOW_PASS = 30.0  

In [39]:
N2pc_no_rejection,df = create_mne_raw(Config.DATA_FILE)
events,event_ids,stimulus_count = extract_events(raw=N2pc_no_rejection)
Config.EPOCH_TMAX = 0.8
Config.EPOCH_TMIN = -0.2
Config.BASELINE = (-0.2,0)
Config.EVENT_DICT = event_ids
epochs = create_epochs(N2pc_no_rejection, events,Config)
# fig, ax = plt.subplots(figsize=[15, 5])

# mne.viz.plot_events(events, N2pc_no_rejection.info['sfreq'],  
#                     event_id=event_ids,                    
#                     axes=ax)
# plt.show()

=== CREATING MNE OBJECT ===
Number of trials detected: 1
=== EXTRACTING EVENTS ===

Found 642 annotations:
  211: 43 occurrences
  212: 37 occurrences
  111: 42 occurrences
  121: 38 occurrences
  222: 43 occurrences
  202: 55 occurrences
  122: 42 occurrences
  221: 37 occurrences
  112: 38 occurrences
  201: 267 occurrences
Used Annotations descriptions: [np.str_('111'), np.str_('112'), np.str_('121'), np.str_('122'), np.str_('201'), np.str_('202'), np.str_('211'), np.str_('212'), np.str_('221'), np.str_('222')]
=== CREATING EPOCHS ===
Not setting metadata
642 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 642 events and 257 original time points ...
0 bad epochs dropped
111: 42
112: 38
121: 38
122: 42
201: 267
202: 55
211: 43
212: 37
221: 37
222: 43
Created epochs - 111: 42, 112: 38, 121: 38, 122: 42, 201: 267, 202: 55, 211: 43, 212: 37, 221: 37, 222: 43


In [36]:
N2pc_no_rejection.plot(channel='HEOG-bipolar')

TypeError: BaseRaw.plot() got an unexpected keyword argument 'channel'. Did you mean 'n_channels'?

In [133]:
epochs,df = create_mne_raw(Config.DATA_FILE)
montage = mne.channels.make_standard_montage('standard_1020')
cleaned_epochs = epochs.copy().drop(np.where(bad_epochs)[0])
bin_1, bin_2 = create_bins(cleaned_epochs.events, {'condition_1':[1,2,7,8],'condition_2':[3,4,5,6]})
# eog_channels = ['HEOG-left', 'HEOG-right', 'VEOG-lower', 'HEOG-bipolar', 'VEOG-uncorrected']
# channel_types = {ch: 'eog' for ch in eog_channels}
# epochs.set_channel_types(channel_types)
# epochs.set_montage(montage, match_case=False)

=== CREATING MNE OBJECT ===
Number of trials detected: 267
Detected epoched data - using mne.io.read_epochs_eeglab()
Dropped 29 epochs: 69, 80, 85, 87, 91, 94, 133, 135, 152, 153, 154, 155, 156, 162, 164, 175, 179, 205, 209, 215, 217, 218, 227, 229, 234, 236, 238, 251, 261
Using provided bin indices


c:\Users\Keerti\Downloads\MySolution\PlayBooks\EegFunctions.py:38: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  epochs = mne.io.read_epochs_eeglab(path, verbose=False)


In [ ]:
# cleaned_epochs = epochs.copy().drop(np.where(bad_epochs)[0])
epochs_left = cleaned_epochs[bin_1]
epochs_right = cleaned_epochs[bin_2]
channel = 31
# Create ERPs
erp_left = epochs_left.average()
erp_right = epochs_right.average()
times=(-200, 800)
# Create difference wave
erp_diff = mne.combine_evoked([erp_left, erp_right], weights=[1, -1])

# Plot Panel A: Both conditions
# fig, axes = plt.subplots(4, 1, figsize=(12, 12))
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

# Panel A: Overlay plot
times_plot = erp_left.times * 1000  # Convert to ms
left_data = erp_left.get_data(picks=channel)[0] * 1e6  # Convert to µV
right_data = erp_right.get_data(picks=channel)[0] * 1e6

axes[0].plot(times_plot, left_data, 'k-', label='BIN1: Left Target', linewidth=1.5)
axes[0].plot(times_plot, right_data, 'r-', label='BIN2: Right Target', linewidth=1.5)
axes[0].axhline(0, color='gray', linestyle='-', alpha=0.3)
axes[0].axvline(0, color='gray', linestyle='--', alpha=0.5)
axes[0].set_xlim(times[0], times[1])
axes[0].set_ylim(-8, 10)
axes[0].set_xticks([0, 200, 400, 600, 800])  # X-axis ticks
axes[0].set_yticks([-8, -6, -4, -2, 0, 2, 4, 6, 8, 10])  # Y-axis ticks
axes[0].set_ylabel('Amplitude (µV)')
axes[0].set_title(f'A    {channel}')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Panel B: Difference wave only
diff_data = erp_diff.get_data(picks=channel)[0] * 1e6
axes[1].plot(times_plot, diff_data, 'b-', label='BIN3: Left Target Minus Right Target', linewidth=1.5)
axes[1].axhline(0, color='gray', linestyle='-', alpha=0.3)
axes[1].axvline(0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlim(times[0], times[1])
axes[1].set_ylim(-8, 10)
axes[1].set_ylabel('Amplitude (µV)')
axes[1].set_title(f'B    {channel}')
axes[1].set_xticks([0, 200, 400, 600, 800])  # X-axis ticks
axes[1].set_yticks([-8, -6, -4, -2, 0, 2, 4, 6, 8, 10])  # Y-axis ticks

axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Panel C: All three traces
axes[2].plot(times_plot, left_data, 'k-', label='BIN1: Left Target', linewidth=1.5)
axes[2].plot(times_plot, right_data, 'r-', label='BIN2: Right Target', linewidth=1.5)
axes[2].plot(times_plot, diff_data, 'b-', label='BIN3: Left Target Minus Right Target', linewidth=1.5)
axes[2].axhline(0, color='gray', linestyle='-', alpha=0.3)
axes[2].axvline(0, color='gray', linestyle='--', alpha=0.5)
axes[2].set_xlim(times[0], times[1])
axes[2].set_ylim(-8, 10)
axes[2].set_ylabel('Amplitude (µV)')
axes[2].set_title(f'C    {channel}')
axes[2].set_xticks([0, 200, 400, 600, 800])  # X-axis ticks
axes[2].set_yticks([-8, -6, -4, -2, 0, 2, 4, 6, 8, 10])  # Y-axis ticks

axes[2].legend()
axes[2].grid(True, alpha=0.3)

# Panel D: Zoomed difference wave
axes[3].plot(times_plot, diff_data, 'b-', label='BIN3: Left Target Minus Right Target', linewidth=1.5)
axes[3].axhline(0, color='gray', linestyle='-', alpha=0.3)
axes[3].axvline(0, color='gray', linestyle='--', alpha=0.5)
axes[3].set_xlim(-200, 300)  # Zoomed time window
axes[3].set_ylim(-3.2, 3.2)  # Zoomed amplitude
axes[3].set_ylabel('Amplitude (µV)')
axes[3].set_xlabel('Time (ms)')
axes[3].set_title(f'D    {channel}')
# axes[3].set_xticks([0, 200, 400, 600, 800])  
axes[3].set_xticks([-200, -100, 0, 100, 200, 300])
axes[3].set_yticks([-3.2, -1.6, 0, 1.6, 3.2])
# a
axes[3].legend()
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()
    

Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
none


In [129]:
bad_epochs = moving_window_step_artifact_detection(epochs, channel_name="HEOG-bipolar",window_size=0.2, step_size=0.01, threshold=16, tmin=-0.2, tmax=0.4 )
np.where(bad_epochs)

(array([ 69,  80,  85,  87,  91,  94, 133, 135, 152, 153, 154, 155, 156,
        162, 164, 175, 179, 205, 209, 215, 217, 218, 227, 229, 234, 236,
        238, 251, 261]),)

In [106]:
plot_bad_epochs(epochs,bad_epochs,'HEOG-bipolar',n_epochs_to_plot=15,title="Blink Artifact Detection")

Using matplotlib as 2D backend.


In [123]:
# list(epochs_small_eye_movement.event_id.keys())
epochs_bc[0,1,6,7]

<EpochsEEGLAB | 4 events (all good), -0.199 – 0.797 s (baseline off), ~305 KiB, data loaded,
 'B1(112)/201': 1
 'B1(111)/201': 1
 'B2(122)/201': 1
 'B2(121)/201': 1>

In [ ]:
epochs_small_eye_movement = epochs.drop(np.where(bad_epochs)[0])
# epoch_lists = [epochs_small_eye_movement]
time_window=(0.2, 0.3)
# sme_results = compute_sme_erplab(epoch_lists,time_window)

import numpy as np

for ep in [epochs_small_eye_movement,epochs]:
    epochs_bc = ep.copy()#.apply_baseline((-0.2, 0))
    ch_name = 'PO7'
    tmin, tmax = time_window[0],time_window[1]  # 125–225 ms window
    mask = (epochs_bc.times >= tmin) & (epochs_bc.times <= tmax)

    bins = list(epochs_bc.event_id.keys())[2::]  
    aSME_dict = {}
    # for b in bins:
    data = epochs_bc[0,1,6,7].get_data()[:, epochs_bc.ch_names.index(ch_name), :] * 1e6  # volts → μV
    trial_means = data[:, mask].mean(axis=1)  # mean per trial in window
    aSME = trial_means.std() / np.sqrt(trial_means.size)  # standard error across trials
    aSME_dict[b] = aSME

    for b, val in aSME_dict.items():
        print(f"aSME {ch_name} {tmin}, {tmax} ms ({b}): {val:.4f} μV")

aSME PO7 0.2, 0.3 ms (B1(211)/201): 3.4105 μV
aSME PO7 0.2, 0.3 ms (B1(211)/201): 3.4105 μV
